In [1]:
import roboticstoolbox as rtb
import numpy as np
from spatialmath import SE3
from spatialmath.base import sym

%matplotlib notebook

<IPython.core.display.Javascript object>

<frozen importlib._bootstrap>:219: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.


The traditional approach to robot kinematics is to use Denavit-Hartenberg notation where each link is deescribed by a very specific sequence of simple (elementary) transformations

$\mathbf{A}_j = r_z(\theta_j) t_z(d_j) t_x(a_j) r_x(\alpha_j)$

some of which may be missing.  The parameters $(\theta_j, d_j, a_j, \alpha_j)$ are the Denavit-Hartenberg parameters.  A complete robot is defined as a sequence of $\mathbf{A}_j$, one per joint, 

$\mathbf{T} = \mathbf{T}_b \prod_j\mathbf{A}_j \mathbf{T}_t$

sometimes with an addition base transform at the befinning and a tool transform on the end.
For a revolute joint $\theta_j = q_j$ is the joint variable, whereas for a prismatic (sliding) joint $d_j = q_j$ is the joint variable.

Because a rigid-link has effectively 6 degrees-of-freedom, using just 4 parameters leads to some constraints.  This all conspires to making the Denavit-Hartenberg notation cumbersome to learn, apply and determine for a new robot.

If we multiply this out we see it is a sequence of transformations drawn from the set $\{r_x, r_z, t_x, t_z\}$.
Let's take a step back and consider defining a robot from a sequence of transforms drawn from the complete set of all possible elementary (or canonical) transformations $\{r_x, r_y, r_z, t_x, t_y, t_z\}$ any of which can be a function of a constant or a joint variable.

The Robotics Toolbox implements this functionality. We will start simply


In [2]:
from roboticstoolbox import ETS as E

E.rx(45, 'deg')

Rx(45°)

In [3]:
E.ty(2)

ty(2)

In [4]:
E.ry()

Ry(q)

Consider we want to create a simple 2 link planar robot arranged like

o-------o--<>---x 

where 'o' is a joint, 'x' is the end effector, the first link is 1m long, and the second link can change its length - it is a prismatic joint.  The robot can be represented by just 4 elementary transforms

In [5]:
e = E.rz() * E.tx(1) * E.rz() * E.tx()
e

Rz(q0) * tx(1) * Rz(q1) * tx(q2)

`e` is an ETS object

In [6]:
type(e)

roboticstoolbox.robot.ETS.ETS

which has a number of methods including 

In [7]:
len(e)

4

indicating it has four transforms

In [10]:
e.n

3

indicating it has three joint variables, and they occur at the indices given by

In [11]:
e.joints()

array([0, 2, 3])

The ETS object acts a lot like a list, and we can slice it

In [12]:
e[1]

tx(1)

In [13]:
e[2:]

Rz(q0) * tx(q1)

There are also a number of predicates

In [14]:
e[0].isjoint

True

In [15]:
e[1].isjoint

False

In [16]:
e[1].isrevolute

False

In [17]:
e[3].isrevolute

False

In [18]:
e[3].isprismatic

True

We can substitute in values

In [19]:
e.eval([0, 0, 1])

SE3:   1           0           0           2            
       0           1           0           0            
       0           0           1           0            
       0           0           0           1            
    

which as expected is a translation of 2m in the x-direction.  Let try another configuration

In [20]:
e.eval([90, -90, 2], 'deg')

SE3:   1           0           0           2            
       0           1           0           1            
       0           0           1           0            
       0           0           0           1            
    

Note that the `'deg'` option only applies to those list elements corresponding to angles.

## Symbolics



In [21]:
q = sym.symbol('q_:3')
print(q)

(q_0, q_1, q_2)


Which creates a vector of three joint angle variables. The advantage of having the underscore in the name is that Jupyter considers this as LaTeX subscript notation when it pretty prints the symbols

In [22]:
q[0]

q_0

We can substitute in the symbolic values just as easily as we did the numerical values to find the resulting overall transform

In [23]:
e.eval(q)

SE3:  -sin(q_0)*sin(q_1) + cos(q_0)*cos(q_1)-sin(q_0)*cos(q_1) - sin(q_1)*cos(q_0)0           q_2*(-sin(q_0)*sin(q_1) + cos(q_0)*cos(q_1)) + cos(q_0)  
      sin(q_0)*cos(q_1) + sin(q_1)*cos(q_0)-sin(q_0)*sin(q_1) + cos(q_0)*cos(q_1)0           q_2*(sin(q_0)*cos(q_1) + sin(q_1)*cos(q_0)) + sin(q_0)  
      0           0           1           0             
      0           0           0           1             
    

## A more complex example

We could consider something more ambitious like a Puma560 robot which six unique lengths

Reference:
- [A simple and systematic approach to assigning Denavit-Hartenberg parameters](https://petercorke.com/robotics/a-simple-and-systematic-approach-to-assigning-denavit-hartenberg-parameters), Peter I. Corke, IEEE Transactions on Robotics, 23(3), pp 590-594, June 2007.

In [24]:
l1 = 0.672;
l2 = 0.2337
l3 = 0.4318
l4 = -0.0837
l5 = 0.4318
l6 = 0.0203

and we can describe the tip of the robot by a sequence of elementary transforms

In [25]:
e = E.tz(l1) * E.rz() * E.ty(l2) * E.ry() * E.tz(l3) * E.tx(l6) * \
    E.ty(l4) * E.ry() * E.tz(l5) * E.rz() * E.ry() * E.rz() * E.tx(0.2)
e

tz(0.672) * Rz(q0) * ty(0.2337) * Ry(q1) * tz(0.4318) * tx(0.0203) * ty(-0.0837) * Ry(q2) * tz(0.4318) * Rz(q3) * Ry(q4) * Rz(q5) * tx(0.2)

In [26]:
e.n

6

In [27]:
e.joints()

array([ 1,  3,  7,  9, 10, 11])

In [28]:
e.eval([0]*6)

SE3:   1           0           0           0.2203       
       0           1           0           0.15         
       0           0           1           1.5356       
       0           0           0           1            
    

We can compute not only forward kinematics, but also the differential kinematics

In [29]:
J = e.jacobe([0]*6)
J

array([[-0.15  ,  0.8636,  0.4318,  0.    ,  0.    ,  0.    ],
       [ 0.2203,  0.    ,  0.    ,  0.2   ,  0.    ,  0.2   ],
       [ 0.    , -0.2203, -0.2   ,  0.    , -0.2   ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  1.    ,  1.    ,  0.    ,  1.    ,  0.    ],
       [ 1.    ,  0.    ,  0.    ,  1.    ,  0.    ,  1.    ]])

which relates the rate of change of the end-effector position to the rate of change of joint coordinates.
The rate of change of the end-effector position is its velocity, but because it moves in 3D we need to describe its translational velocity (a 3-vector) and its angular velocity (another 3-vector).  We pack these into a 6-vector which is known as _spatial velocity_ and denoted by $\nu$.  These quantities are related by

$\nu = \mathbf{J}(q) \dot{q}$

The Hessian is a tensor (3-dimensional matrix) that relates joint velocity to end-effector acceleration $\dot{\nu}$

In [30]:
e.hessian0([0]*6)

array([[[-0.2203, -0.    , -0.    , -0.2   , -0.    , -0.2   ],
        [-0.    , -0.2203, -0.2   ,  0.    , -0.2   ,  0.    ],
        [-0.    , -0.2   , -0.2   ,  0.    , -0.2   ,  0.    ],
        [-0.2   ,  0.    ,  0.    , -0.2   , -0.    , -0.2   ],
        [-0.    , -0.2   , -0.2   , -0.    , -0.2   ,  0.    ],
        [-0.2   ,  0.    ,  0.    , -0.2   ,  0.    , -0.2   ]],

       [[-0.15  ,  0.8636,  0.4318,  0.    ,  0.    ,  0.    ],
        [ 0.8636,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
        [ 0.4318,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ]],

       [[ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
        [ 0.    , -0.8636, -0.4318,  0.    ,  0.    ,  0.    ],
        [ 0.    , -0.4318, -0.4318,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , 

## Compilation

If we were to evaluate this ETS many times we would be peforming lots of unneccessary multiplication of constant terms such as 

In [31]:
e[4:7]

tz(0.4318) * tx(0.0203) * ty(-0.0837)

To make an ETS more efficient to run we can "compile it"

In [32]:
ec = e.compile()
ec

C0 * Rz(q0) * C1 * Ry(q1) * C2 * Ry(q2) * C3 * Rz(q3) * Ry(q4) * Rz(q5) * C4

and we can see that consecutive constant transforms have been folded into new "not so elementary" transforms denoted by `Ci`.  The value of the ETS, is of course the same.

In [33]:
ec.eval([0]*6)

SE3:   1           0           0           0.2203       
       0           1           0           0.15         
       0           0           1           1.5356       
       0           0           0           1            
    

## Handling joint offsets

When using Denavit-Hartenberg notation, we are often forced to choose a robot joint configuration for zero joint angles that is not what the controller or the user may wish to define as zero joint angles.  The `DHRobot` allows the user to specify joint coordinate offsets, a per-link parameter stored in the robots `DHLink` subclass elements.  This is subtracted from the user's joint angles to obtain the kinematic joint angles on which we compute forward kinematics or Jacobians.  Similarly, these offsets are added to the results of any inverse kinematic solution.

With the ETS convention things are a lot easier. We just draw (or imagine) the robot in the pose we define as the "zero angle pose" - the configuration where all the joint coordinates are zero.  It might be that this leads to transforms sequences with consecutive elements which are a constant and joint transform about the same axis, for example  `rx(90, 'deg') rx()`.  We could write this in arbitrary order - the result will be the same - but for the toolbox we adopt the convention that a constant transformation on the same axis as a joint is placed before the joint transform.  

The justification for this is that  because when wen come to assign link frames we place them immediately after a joint transform.  For the Puma560 case


**{0}** E.tz(l1) * E.rz() **{1}** * E.ty(l2) * E.ry() **{2}** * E.tz(l3) * E.tx(l6) * \
    E.ty(l4) * E.ry() **{3}** * E.tz(l5) * E.rz() **{4}** * E.ry() **{5}** * E.rz() **{6}** * E.tx(0.2) **{ee}**


These extra transformation incur minimal computational overhead, since after compilation, they will be folded in with other constant transforms.


## Converting an ETS to a robot

The ETS succinctly describes the forward kinematics of a robot and we can also compute its Jacobian and Hessian.  However Toolbox robot objects have additional capability for graphics, inverse kinematics and dynamics.  We can _promote_ an ETS to a robot by

In [34]:
robot = rtb.ERobot(e)
print(robot)

┌──────┬────────┬────────────────────────────────────────────────┐
│ link │ parent │                      ETS                       │
├──────┼────────┼────────────────────────────────────────────────┤
│link0 │      - │                             tz(0.672) * Rz(q0) │
│link1 │  link0 │                            ty(0.2337) * Ry(q1) │
│link2 │  link1 │ tz(0.4318) * tx(0.0203) * ty(-0.0837) * Ry(q2) │
│link3 │  link2 │                            tz(0.4318) * Rz(q3) │
│link4 │  link3 │                                         Ry(q4) │
│link5 │  link4 │                                         Rz(q5) │
│   ee │  link5 │                                        tx(0.2) │
└──────┴────────┴────────────────────────────────────────────────┘



The ETS has been chopped into segments that connect links.  Each link is a rigid-body with an attached coordinate frame.  Relative to that frame is a short ETS that ends with joint transform that describes the coordinate frame of the next link in the chain.

Note that the last link, with its name in blue, has no joint transform.  It is simply a constant relative pose with respect to the link 5 coordinate frame.

## Converting a robot to ETS

We can also perform the inverse transformation. Consider a model defined using Denavit-Hartenberg notation

In [35]:
puma = rtb.models.DH.Puma560()
print(puma)

┏━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓
┃θⱼ  ┃   dⱼ    ┃   aⱼ   ┃   ⍺ⱼ   ┃   q⁻    ┃   q⁺   ┃
┣━━━━╋━━━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━━━╋━━━━━━━━┫
┃ q1 ┃       0 ┃      0 ┃  90.0° ┃ -160.0° ┃ 160.0° ┃
┃ q2 ┃       0 ┃ 0.4318 ┃   0.0° ┃ -110.0° ┃ 110.0° ┃
┃ q3 ┃ 0.15005 ┃ 0.0203 ┃ -90.0° ┃ -135.0° ┃ 135.0° ┃
┃ q4 ┃  0.4318 ┃      0 ┃  90.0° ┃ -266.0° ┃ 266.0° ┃
┃ q5 ┃       0 ┃      0 ┃ -90.0° ┃ -100.0° ┃ 100.0° ┃
┃ q6 ┃       0 ┃      0 ┃   0.0° ┃ -266.0° ┃ 266.0° ┃
┗━━━━┻━━━━━━━━━┻━━━━━━━━┻━━━━━━━━┻━━━━━━━━━┻━━━━━━━━┛

┌─────┬─────┬──────┬───────┬─────┬──────┬─────┐
│name │ q0  │ q1   │ q2    │ q3  │ q4   │ q5  │
├─────┼─────┼──────┼───────┼─────┼──────┼─────┤
│  qz │  0° │  0°  │  0°   │  0° │  0°  │  0° │
│  qr │  0° │  90° │ -90°  │  0° │  0°  │  0° │
│  qs │  0° │  0°  │ -90°  │  0° │  0°  │  0° │
│  qn │  0° │  45° │  180° │  0° │  45° │  0° │
└─────┴─────┴──────┴───────┴─────┴──────┴─────┘



In [36]:
print(puma.ets())

Rx(90°) * Rz(q0) * tx(0.4318) * Rz(q1) * tx(0.0203) * Rx(-90°) * Rz(q2) * tz(0.15005) * Rx(90°) * Rz(q3) * tz(0.4318) * Rx(-90°) * Rz(q4) * Rz(q5)


Or for a robot with a prismatic joint

In [37]:
stanford = rtb.models.DH.Stanford()
print(stanford)

┏━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃  θⱼ   ┃  dⱼ   ┃   aⱼ   ┃   ⍺ⱼ   ┃         q⁻          ┃   q⁺   ┃
┣━━━━━━━╋━━━━━━━╋━━━━━━━━╋━━━━━━━━╋━━━━━━━━━━━━━━━━━━━━━╋━━━━━━━━┫
┃ q1    ┃ 0.412 ┃      0 ┃ -90.0° ┃             -170.0° ┃ 170.0° ┃
┃ q2    ┃ 0.154 ┃    0.0 ┃  90.0° ┃             -170.0° ┃ 170.0° ┃
┃-90.0° ┃    q3 ┃ 0.0203 ┃   0.0° ┃ 0.30479999999999996 ┃   1.27 ┃
┃ q4    ┃     0 ┃      0 ┃ -90.0° ┃             -170.0° ┃ 170.0° ┃
┃ q5    ┃     0 ┃      0 ┃  90.0° ┃              -90.0° ┃  90.0° ┃
┃ q6    ┃     0 ┃      0 ┃   0.0° ┃             -170.0° ┃ 170.0° ┃
┗━━━━━━━┻━━━━━━━┻━━━━━━━━┻━━━━━━━━┻━━━━━━━━━━━━━━━━━━━━━┻━━━━━━━━┛

┌─────┬─────┬─────┬────┬─────┬─────┬─────┐
│name │ q0  │ q1  │ q2 │ q3  │ q4  │ q5  │
├─────┼─────┼─────┼────┼─────┼─────┼─────┤
│  qz │  0° │  0° │  0 │  0° │  0° │  0° │
└─────┴─────┴─────┴────┴─────┴─────┴─────┘



In [38]:
print(stanford.ets())

Rx(-90°) * Rz(q0) * tz(0.412) * Rx(90°) * Rz(q1) * tz(0.154) * tx(0.0203) * tz(-1.5707963267948966) * tz(q2) * Rx(-90°) * Rz(q3) * Rx(90°) * Rz(q4) * Rz(q5)
